In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import os
import sys

In [2]:
sys.path.append(os.path.abspath('../scripts')) 
model = joblib.load("../models/Next_Tmin_Forecast_Final.pkl")

In [3]:
# Load dữ liệu
df = pd.read_csv("../data/processed/external_weather_processed.csv")

df.drop(['datetime'], axis=1, inplace=True)

y_true = df['target_tempmin'] 
df.drop(['target_tempmin'], axis=1, inplace=True)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = scaler.fit_transform(df)
y_pred = model.predict(df) 


mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"MAE: {mae:.3f}")
print(f"MSE: {mse:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"R² Score: {r2:.3f}")

# Tạo DataFrame
df_Predicted = pd.DataFrame({
    "Actual Values": y_true,
    "Predicted Values": y_pred
}, index=range(len(y_true)))

df_Predicted["Sai lệch"] = df_Predicted["Actual Values"] - df_Predicted["Predicted Values"]
# Đếm số trường có độ lệch lớn hơn 2 °C (dựa trên giá trị tuyệt đối)
large_deviation_count = len(df_Predicted[abs(df_Predicted["Sai lệch"]) > 2])
print(f"\nSố trường có độ lệch lớn hơn 2 °C (target_tempmin): {large_deviation_count}")
print(f"Tỷ lệ: {large_deviation_count / len(df_Predicted) * 100:.2f}%")

# Hiển thị toàn bộ DataFrame
print("\nDataFrame với giá trị thực tế và dự đoán trên thang gốc:")
print(df_Predicted)

# Đặt lại thiết lập hiển thị về mặc định sau khi in
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')


MAE: 1.157
MSE: 2.231
RMSE: 1.494
R² Score: 0.908

Số trường có độ lệch lớn hơn 2 °C (target_tempmin): 112
Tỷ lệ: 16.07%

DataFrame với giá trị thực tế và dự đoán trên thang gốc:
     Actual Values  Predicted Values  Sai lệch
0             10.3         14.160255 -3.860255
1             11.4         13.044697 -1.644697
2             14.0         13.016678  0.983322
3             14.1         13.328966  0.771034
4             14.4         15.388452 -0.988452
..             ...               ...       ...
692           14.4         15.309626 -0.909626
693           10.7         14.328909 -3.628909
694           11.8         15.262840 -3.462840
695           14.5         15.784456 -1.284456
696           12.3         15.586521 -3.286521

[697 rows x 3 columns]


# <b> Đánh giá mô hình và so sánh với bộ train </b>

### 1. Tổng quan

Để đánh giá hiệu quả của mô hình dự đoán nhiệt độ nhỏ nhất trên tập kiểm tra (test set) và so sánh với kết quả trên tập huấn luyện (train set), nhóm sử dụng các chỉ số phổ biến: `MAE`, `MSE`, `RMSE`, và `R² Score`. Phân tích sai lệch giữa giá trị thực tế và dự đoán cũng được thực hiện để đánh giá khả năng tổng quát của mô hình và kiểm tra hiện tượng **overfitting**. Kết quả trên tập kiểm tra (696 mẫu) sẽ được so sánh với kết quả trên tập huấn luyện (1.701 mẫu) để đưa ra kết luận.

### 2. Chỉ số đánh giá mô hình trên tập kiểm tra

Sau khi đánh giá mô hình trên tập kiểm tra (696 mẫu), các chỉ số thu được như sau:

| **Chỉ số** | **Tập huấn luyện (1.701 mẫu)** | **Tập kiểm tra (696 mẫu)** | **So sánh** |
|------------|-------------------------------|---------------------------|-------------|
| MAE        | 1.047°C                      | 1.157°C                   | Tăng 0.110°C (~10.5%) |
| MSE        | 1.822°C²                     | 2.231°C²                  | Tăng 0.409°C² (~22.4%) |
| RMSE       | 1.350°C                      | 1.494°C                   | Tăng 0.144°C (~10.7%) |
| R² Score   | 0.930                        | 0.908                     | Giảm 0.022 (~2.4%) |
| Số mẫu sai lệch >2°C | 221 (~13.0%)          | 112 (~16.07%)             | Tăng tỷ lệ ~3.07% |

##### 2.1. MAE
`MAE` ≈ 1.157°C: trung bình các dự đoán trên tập kiểm tra lệch khoảng 1.16°C so với giá trị thực tế.
  - Ví dụ từ DataFrame:
    - Dòng 0: Thực tế = 10.3°C, dự đoán = 14.160255°C, sai số = -3.860255°C (sai số lớn).
    - Dòng 2: Thực tế = 14.0°C, dự đoán = 13.016678°C, sai số = 0.983322°C (sai số nhỏ).  

**Nhận xét**:  MAE ≈ 1.157°C là mức chấp nhận được trong dự báo nhiệt độ nhỏ nhất, nhưng cao hơn so với tập huấn luyện (MAE ≈ 1.047°C). Điều này cho thấy mô hình hoạt động kém hơn một chút trên dữ liệu mới, nhưng vẫn trong ngưỡng phù hợp cho dự báo thời tiết thông thường.  
##### 2.2. MSE
`MSE` ≈ 2.231°C² cho thấy có một số sai số lớn (ví dụ, dòng 0: sai số -3.860255°C, bình phương ≈ 14.90°C²), nhưng tổng thể sai số vẫn được kiểm soát.

**Nhận xét**: MSE trên tập kiểm tra (2.231°C²) cao hơn so với tập huấn luyện (1.822°C²), cho thấy mô hình có sai số lớn hơn trên dữ liệu mới, đặc biệt trong các trường hợp thời tiết bất thường.

##### 2.3. Root Mean Squared Error - RMSE
`RMSE` ≈ 1.494°C cao hơn MAE, cho thấy có các sai số lớn (như dòng 0: -3.860255°C hoặc dòng 694: -3.462840°C) làm tăng giá trị.

**Nhận xét**: RMSE ≈ 1.494°C cao hơn so với tập huấn luyện (1.350°C), cho thấy mô hình có xu hướng sai lệch lớn hơn trên tập kiểm tra. Mặc dù vẫn chấp nhận được trong dự báo thời tiết, nhưng điều này gợi ý rằng mô hình cần cải thiện để giảm sai số lớn trong các ứng dụng dự đoán nhiệt độ.

##### 2.4. R² Score
`R²` ≈ 0.908 cho thấy mô hình giải thích được 90.8% sự biến thiên của nhiệt độ nhỏ nhất trên tập kiểm tra, là kết quả tốt nhưng thấp hơn so với tập huấn luyện (0.930).

**Nhận xét**: R² ≈ 0.908 vẫn là một giá trị cao, cho thấy mô hình có khả năng dự đoán tốt trên dữ liệu mới. Tuy nhiên, sự giảm nhẹ so với tập huấn luyện (0.930) cho thấy mô hình có thể không tổng quát hoàn toàn trên các điều kiện thời tiết mới.

### 3. Phân tích sai số và độ lệch trên tập kiểm tra
- **Tổng số mẫu kiểm tra**: 696
- **Số mẫu có sai lệch lớn hơn 2°C**: 112 mẫu (~16.07%)
- **Tỷ lệ dự đoán chính xác (sai lệch ≤ 2°C)**: ~83.93%

**Nhận xét**:  
- Tỷ lệ 16.07% điểm có sai số >2°C (112/696) cao hơn so với tập huấn luyện (13.0%, 221/1.701), cho thấy mô hình gặp khó khăn hơn khi dự đoán trên dữ liệu mới. Tuy nhiên, 83.93% điểm có sai số ≤2°C vẫn là kết quả tích cực.
- Ví dụ sai số lớn:
  - Dòng 0 (10.3°C thực tế, 14.160255°C dự đoán): Sai số = -3.860255°C. Mô hình có thể đã đánh giá quá cao nhiệt độ nhỏ nhất do các đặc trưng như độ che phủ mây hoặc độ ẩm không được cân bằng tốt.
  - Dòng 694 (11.8°C thực tế, 15.262840°C dự đoán): Sai số = -3.462840°C. Tương tự, mô hình có thể đã dự đoán nhiệt độ cao hơn do không nắm bắt tốt các yếu tố làm mát (như gió mạnh hoặc độ ẩm thấp).

### 4. So sánh với kết quả trên tập huấn luyện

Để đánh giá xem mô hình có bị **overfitting** hay không, chúng ta so sánh các chỉ số giữa tập huấn luyện và tập kiểm tra:

**Phân tích so sánh**:
- MAE, MSE, RMSE tăng trên tập kiểm tra: Các chỉ số sai số trên tập kiểm tra cao hơn tập huấn luyện (MAE tăng 10.5%, RMSE tăng 10.7%, MSE tăng 22.4%). Điều này cho thấy mô hình có hiệu suất thấp hơn trên dữ liệu mới nhưng mức tăng không quá lớn, vẫn trong ngưỡng chấp nhận được.

- R² giảm nhẹ: R² giảm từ 0.930 xuống 0.908 (2.4%) tuy nhiên vẫn ở mức cao (> 0.9), cho thấy mô hình vẫn giải thích tốt sự biến thiên của nhiệt độ nhỏ nhất trên tập kiểm tra, dù kém hơn một chút so với tập huấn luyện.

- Tỷ lệ sai số >2°C tăng: Tỷ lệ điểm có sai số >2°C tăng từ 13.0% (tập huấn luyện) lên 16.07% (tập kiểm tra). Điều này cho thấy mô hình gặp khó khăn hơn trong việc dự đoán chính xác trên dữ liệu mới, đặc biệt trong các điều kiện thời tiết phức tạp.

**Kết luận về overfitting**:
- Mô hình không bị overfitting vì sự khác biệt giữa các chỉ số trên tập huấn luyện và tập kiểm tra là nhỏ (MAE tăng ~0.11°C, R² giảm ~0.022). Sự suy giảm hiệu suất trên tập kiểm tra là bình thường, vì dữ liệu mới thường có các mẫu thời tiết khác biệt so với dữ liệu huấn luyện.

- Tuy nhiên, sự gia tăng tỷ lệ sai số >2°C (từ 13.0% lên 16.07%) và các chỉ số sai số cao hơn cho thấy mô hình có thể chưa tổng quát hóa tốt trong các điều kiện thời tiết bất thường (ví dụ, dòng 0 và 694 trên tập kiểm tra có sai số lớn >3°C).

### 5. Kết luận

Mô hình dự đoán nhiệt độ nhỏ nhất hoạt động tốt trên cả tập huấn luyện và tập kiểm tra, với MAE ≈ 1.157°C, RMSE ≈ 1.494°C, và R² ≈ 0.908 trên tập kiểm tra, cho thấy khả năng dự đoán chính xác trong phần lớn trường hợp. Tuy nhiên, hiệu suất trên tập kiểm tra thấp hơn một chút so với tập huấn luyện, đặc biệt trong các điều kiện thời tiết phức tạp, nơi các đặc trưng như `cloudcover`, `windspeed`, và `humidity` có tương tác phi tuyến. Mô hình không bị overfitting nghiêm trọng, nhưng vẫn còn khoảng 16.07% điểm có sai số >2°C trên tập kiểm tra, cho thấy cần cải thiện để tăng độ chính xác.
